In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import emoji
from sklearn.metrics import f1_score
import numpy as np

In [2]:


data = pd.read_csv('Daraz Reviews.csv')
data= data.sample(frac=1, random_state=42)

In [3]:

import re
import emoji
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer

# Define a function to clean the text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuations
    text = re.sub('[^a-z\s]', '', text)
    # Convert emoji to text
    text = emoji.demojize(text)
    # Perform POS tagging
    tokens = text.split()
    tagged_tokens = pos_tag(tokens)
    pos_tokens = [tag[0] + '_' + tag[1] for tag in tagged_tokens]
    text = ' '.join(pos_tokens)
    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

# applying the function to convert text into more useful form

data['Reviews'] = data['Reviews'].apply(clean_text)
distinct_labels = data['Sentiments'].unique()

print(distinct_labels)
data['Sentiments'] = data['Sentiments'].str.lower()
valid_sentiments = ['positive', 'neutral', 'negative']
data = data[data['Sentiments'].isin(valid_sentiments)]

distinct_labels = data['Sentiments'].unique()

print(distinct_labels)

['Neutral' 'positive' 'neutral' 'Positive' 'negative' 'Negative'
 'Negative ' 'posiitve' 'Negtaive' 'neutal']
['neutral' 'positive' 'negative']


In [4]:
# define a dictionary that maps the old values to the new ones
sentiment_mapping = {'neutral': 0, 'negative': 1, 'positive': 2}

# apply the mapping to the 'Sentiment' column
data['Sentiments'] = data['Sentiments'].map(sentiment_mapping)
# drop rows with NaN values in the 'Sentiment' column
data = data.dropna(subset=['Sentiments'])

# convert the 'Sentiment' column to integer
data['Sentiments'] = data['Sentiments'].astype(int)

In [5]:
df = data

In [6]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X = df['Reviews']
y = df['Sentiments']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# text data into sequences of integers using a tokenizer and pads or truncates the sequences to a fixed length of 200.

tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Then, we pad our sequences so they're all the same length
X_train_pad = pad_sequences(X_train_seq, maxlen=200)
X_test_pad = pad_sequences(X_test_seq, maxlen=200)

# Concatenate train and test data
all_data = np.concatenate((X_train_pad, X_test_pad), axis=0)

# Convert the concatenated data to a dataframe
df = pd.DataFrame(all_data)

# Save the dataframe to a CSV file
df.to_csv('transformed_data.csv', index=False)


In [9]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

# different tyoes of layers are used with Conv1D with pooling layer
embed_dim = 128

model_cnn = Sequential()
model_cnn.add(Embedding(2000, embed_dim, input_length = X_train_pad.shape[1]))
model_cnn.add(SpatialDropout1D(0.4))
model_cnn.add(Conv1D(filters=512, kernel_size=10, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dense(3, activation='softmax'))  # 3 classes for sentiment

from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from sklearn.metrics import classification_report

# ...

model_cnn.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

model_cnn.fit(X_train_pad, y_train, epochs=3, batch_size=64, validation_split=0.2)

# Make predictions on the test set
y_pred = model_cnn.predict(X_test_pad)

# Convert the predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate classification report
report = classification_report(y_test, y_pred_classes)
print('Classification Report:')
print(report)

# Calculate F1 score
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print('Test F1 Score: %f' % (f1*100))

loss, accuracy = model_cnn.evaluate(X_test_pad, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy*100))


Epoch 1/3
87/87 [==============================] - 67s 743ms/step - loss: 0.7264 - accuracy: 0.6804 - val_loss: 0.4270 - val_accuracy: 0.8310
Epoch 2/3
87/87 [==============================] - 60s 690ms/step - loss: 0.3626 - accuracy: 0.8655 - val_loss: 0.3817 - val_accuracy: 0.8622
Epoch 3/3
54/54 [==============================] - 5s 83ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.63      0.67       269
           1       0.85      0.91      0.88       498
           2       0.94      0.94      0.94       957

    accuracy                           0.88      1724
   macro avg       0.84      0.83      0.83      1724
weighted avg       0.88      0.88      0.88      1724

Test F1 Score: 88.057214
Test Accuracy: 88.283062


In [ ]:
import pickle
 
# Save the model as a pickle file
with open('trained_model2.pkl', 'wb') as file:
    pickle.dump(model_cnn, file)
print('Model saved successfully.')

Model saved successfully.


In [ ]:
import pandas as pd
import pickle


# testing on the data provided by mam

file_path = 'test-data.txt'

data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        sentiment, review = line.strip().split('\t')
        data.append([sentiment, review])


df2 = pd.DataFrame(data, columns=['Sentiment', 'Review'])

# Replace tab delimiter with comma
df2['Review'] = df2['Review'].str.replace('\t', ',')

# Print the DataFrame
print(df2)




    Sentiment                                             Review
0    positive  Thank you seller and thank you Daraz for your ...
1    positive  It is exactly as advertised. The finish is smo...
2    positive  They are extremely thin and is so helpful in p...
3    positive    gud quality..i ordered 2 bags..really satisfied
4    positive  Delivered on time . Highly satisfied with prod...
..        ...                                                ...
145   neutral  The mattress is perfect however it came wrappe...
146   neutral  vibrant colours..but..the  material is not cot...
147   neutral  i get my order and it's very nice product but ...
148   neutral                                 reasonable product
149   neutral  Product quality is barely satisfactory but hea...

[150 rows x 2 columns]


In [ ]:
df2['Review'] = df2['Review'].apply(clean_text)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

sequences = tokenizer.texts_to_sequences(df2['Review'])
padded_sequences = pad_sequences(sequences, maxlen=200)  
predictions = model_cnn.predict(padded_sequences)
predicted_sentiments = np.argmax(predictions, axis=1)

sentiment_mapping = {'neutral': 0, 'negative': 1, 'positive': 2}
df2['Sentiment'] = df2['Sentiment'].map(sentiment_mapping)
df2['Predicted Sentiment'] = predicted_sentiments

f1 = f1_score(df2['Sentiment'], df2['Predicted Sentiment'], average='weighted')
print('F1 Score: {:.2f}%'.format(f1 * 100))
accuracy = accuracy_score(df2['Sentiment'], df2['Predicted Sentiment'])
print('Accuracy: {:.2f}%'.format(accuracy * 100))

# Print the DataFrame with predicted sentiments
print(df2)


5/5 [==============================] - 0s 86ms/step
F1 Score: 87.25%
Accuracy: 87.33%
     Sentiment                                             Review  \
0            2  thank_NN you_PRP seller_VBP and_CC thank_VBP y...   
1            2  it_PRP is_VBZ exactly_RB as_IN advertised_VBN ...   
2            2  they_PRP are_VBP extremely_RB thin_JJ and_CC i...   
3            2  gud_NN qualityi_NN ordered_VBD bagsreally_RB s...   
4            2  delivered_VBN on_IN time_NN highly_RB satisfie...   
..         ...                                                ...   
145          0  the_DT mattress_NN is_VBZ perfect_JJ however_R...   
146          0  vibrant_JJ coloursbutthe_NN material_NN is_VBZ...   
147          0  i_NN get_VBP my_PRP$ order_NN and_CC its_PRP$ ...   
148          0                           reasonable_JJ product_NN   
149          0  product_NN quality_NN is_VBZ barely_RB satisfa...   

     Predicted Sentiment  
0                      2  
1                      2  
2   

In [ ]:
df2['Predicted Sentiment'].to_csv('predicted_sentiments.csv', index=False)